In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d


from torch.utils.data import ConcatDataset, DataLoader, Subset
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from Helpers.data import SDDataset, PointCloudDataset
# Hyperparameters
input_dim = 3
latent_dim = 512
hidden_dim = 256
learning_rate = 0.001
num_epochs = 100

if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f'Using: {device}')

Using: cuda


In [2]:
d = SDDataset('../Data/ModelNet40', '../Data/sampled_points', device, 1024, 'test')

In [4]:
torch.save(d, '../Data/sd_dataset_1024_test.pt')

In [5]:
trainset = torch.load('../Data/sd_dataset_1024_train.pt', weights_only= False)
testset = torch.load('../Data/sd_dataset_1024_test.pt', weights_only= False)

batch_size = 32 

train_loader = DataLoader(trainset, batch_size= batch_size, shuffle= True)
test_loader = DataLoader(testset, batch_size= batch_size, shuffle= True)


In [6]:
class SD_Model(nn.Module):

    def __init__(self, latent_rep_size):
        
        super().__init__()
        input_size = 3 + latent_rep_size
        self.l1 = nn.Linear(input_size, 1024)
        self.l2 = nn.Linear(1024, 2048)
        self.l3 = nn.Linear(2048, 1024)
        self.l4 = nn.Linear(1024, 128)
        self.l5 = nn.Linear(128, 1)

    def forward(self, latent_rep, xyz):
        x = torch.concat((latent_rep, xyz), dim = 1)
        x = F.gelu(self.l1(x))
        x = F.gelu(self.l2(x))
        x = F.gelu(self.l3(x))
        x = F.gelu(self.l4(x))
        x = self.l5(x)

        return x
    

In [7]:
def train_model(key, model, num_epochs, train_loader, val_loader):

    optimizer = optim.AdamW(model.parameters(), lr=0.0001)

    min_val_loss = np.inf

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):

        # Train one epoch
        train_loss = 0 

        for data in tqdm(train_loader):
            
            latent_rep = data['latent_rep'].to(device)
            xyz = data['xyz'].to(device)
            sd = data['sd'].to(device)

            pred = model(latent_rep, xyz)
            pred = pred.squeeze()

            optimizer.zero_grad()

            loss = F.mse_loss(sd, pred)

            loss.backward()

            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Calculate validation loss

        val_loss = 0 

        for data in val_loader:

            latent_rep = data['latent_rep'].to(device)
            xyz = data['xyz'].to(device)
            sd = data['sd'].to(device)

            with torch.no_grad():
                pred = model(latent_rep, xyz)
                pred = pred.squeeze()

                loss = F.mse_loss(sd, pred)
                val_loss += loss.item()

        val_loss /= len()

        print(f'\nEpoch {epoch+1} \t Train Loss: {train_loss:.5f} \t Val Loss: {val_loss:.5f}')

        # Save best model
        if val_loss < min_val_loss:
            # print(f'Val Loss Decreased({min_val_loss:.6f} ---> {val_loss:.6f}) \t Saving The Model')
            min_val_loss = val_loss

            torch.save(model.state_dict(), f'./trained_autoencoders/{key}')

        train_losses.append(train_loss)
        val_losses.append(val_loss)

    return train_losses, val_losses

model = SD_Model(256)
model = model.to(device)

num_epochs = 5

train_losses, val_losses = train_model("mlp", model, num_epochs , train_loader, test_loader)


  1%|          | 22026/3072813 [00:43<1:28:35, 573.96it/s]

: 

In [15]:
model = SD_Model(256)
model = model.to(device)

num_epochs = 5

train_losses, val_losses = train_model("mlp", model, num_epochs , train_loader, test_loader)

  0%|          | 0/3072813 [00:03<?, ?it/s]


RuntimeError: Boolean value of Tensor with more than one value is ambiguous